In [3]:
import os
import json
import time
import requests
from pathlib import Path
from tqdm import tqdm

# === API setup ===
api_key = "apikey"  # 👈 Insert your API key here
api_url = "https://api.anthropic.com/v1/messages"
model_name = "claude-3-haiku-20240307"

headers = {
    "x-api-key": api_key,
    "anthropic-version": "2023-06-01",
    "content-type": "application/json",
}

# === Paths ===
base_path = Path.home() / "Desktop" / "processed_subs"
prompts_dir = base_path / "prompts_claude_new"  # using the newprompt folder
output_dir = base_path / "scored_prompts_claude_new"  # new output folder
output_dir.mkdir(parents=True, exist_ok=True)

# === Helper: send prompt to Claude and get response ===
def call_claude(prompt):
    payload = {
        "model": model_name,
        "max_tokens": 512,
        "temperature": 0,
        "system": "You are a political sentiment scorer. Always respond ONLY with the exact JSON format requested.",
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }
    
    response = requests.post(api_url, headers=headers, json=payload)
    
    if response.status_code != 200:
        print(f"⚠️ API Error {response.status_code}: {response.text}")
        return None
    
    data = response.json()
    content = data["content"][0]["text"]
    
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        print(f"⚠️ JSON parsing error: {content}")
        return None

# === Main Scoring Loop ===
prompt_files = list(prompts_dir.glob("*.json"))
print(f"Found {len(prompt_files)} prompt files.")

for prompt_file in tqdm(prompt_files, desc="Scoring movies"):
    filename_stem = prompt_file.stem.replace("_prompts", "")
    output_path = output_dir / f"{filename_stem}_scored.json"
    
    # === Check if already scored ===
    if output_path.exists():
        print(f"⚠️ {filename_stem} already scored, skipping...")
        continue
    
    with open(prompt_file, "r", encoding="utf-8") as f:
        prompts_data = json.load(f)
    
    scored_chunks = []
    
    for entry in tqdm(prompts_data, desc=f"Processing {filename_stem}", leave=False):
        chunk_id = entry["chunk_id"]
        prompt = entry["prompt"]
        
        result = call_claude(prompt)
        
        if result is None:
            print(f"❌ Failed on chunk {chunk_id} in {filename_stem}")
            continue
        
        try:
            scored_chunks.append({
                "chunk_id": chunk_id,
                "universalism_score": result["universalism"][0],
                "universalism_confidence": result["universalism"][1],
                "egalitarianism_score": result["egalitarianism"][0],
                "egalitarianism_confidence": result["egalitarianism"][1],
                "progress_score": result["progress"][0],
                "progress_confidence": result["progress"][1],
            })
        except Exception as e:
            print(f"⚠️ Error parsing chunk {chunk_id}: {e}")
            continue
        
        time.sleep(1.5)  # Pause to respect rate limits (adjust if needed)

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(scored_chunks, f, indent=2)
    
    print(f"✅ Finished: {filename_stem} ({len(scored_chunks)} chunks scored)")

print("\n🎉 All done! New movies scored, no overwrites happened!")


Found 443 prompt files.


Scoring movies:   0%|                                   | 0/443 [00:00<?, ?it/s]

⚠️ 1977_Looking.For.Mr.Goodbar.ENG.PAL.en already scored, skipping...
⚠️ 1983_Octopussy.1983.DVD5.720p.HDTV.x264-NBS.eng already scored, skipping...
⚠️ 2020_Birds.of.Prey.And.the.Fantabulous.Emancipation.of.One.Harley.Quinn.2020.HC.720p.HDRip.800M already scored, skipping...
⚠️ 1978_Every.Which.Way.But.Loose.1978.1080p.BluRay.x264-aAF.ENG already scored, skipping...
⚠️ 2002_Signs.2002.720p.BluRay.x264.DTS-HDChina.eng already scored, skipping...
⚠️ 1999_Runaway Bride (1999) [BluRay] [1080p] [YTS.AM].eng already scored, skipping...
⚠️ 1986_Ferris.Buellers.Day.Off.1986.1080p.BluRay.x264-CiNEFiLE.ENG already scored, skipping...
⚠️ 2005_The.Chronicles.Of.Narnia.2005.BluRay.1080p.DTS.x264.dxva-EuReKA.ENG already scored, skipping...
⚠️ 2011_Thor already scored, skipping...
⚠️ 2017_Despicable. Me. 3. 2017. 720p. BluRay. X264-AMIABLE already scored, skipping...
⚠️ 2007_Ratatouille.2007.720p.BluRay.DTS.x264-ESiR.ENG already scored, skipping...
⚠️ 1996_Ransom_EN already scored, skipping...
⚠️ 200


Processing 1977_The Turning Point 1977 720p BRRip x264-PLAYNOW:   0%| | 0/7 [00:
Processing 1977_The Turning Point 1977 720p BRRip x264-PLAYNOW:  14%|▏| 1/7 [00:
Processing 1977_The Turning Point 1977 720p BRRip x264-PLAYNOW:  29%|▎| 2/7 [00:
Processing 1977_The Turning Point 1977 720p BRRip x264-PLAYNOW:  43%|▍| 3/7 [00:
Processing 1977_The Turning Point 1977 720p BRRip x264-PLAYNOW:  57%|▌| 4/7 [00:
Processing 1977_The Turning Point 1977 720p BRRip x264-PLAYNOW:  71%|▋| 5/7 [00:
Processing 1977_The Turning Point 1977 720p BRRip x264-PLAYNOW:  86%|▊| 6/7 [00:
Processing 1977_The Turning Point 1977 720p BRRip x264-PLAYNOW: 100%|█| 7/7 [00:
Scoring movies:  16%|████                      | 70/443 [00:23<02:02,  3.04it/s]

✅ Finished: 1977_The Turning Point 1977 720p BRRip x264-PLAYNOW (7 chunks scored)


...

✅ Finished: 2002_Star.Wars.Episode.II.Attack.of.the.Clones.(2002).DVDRip.XviD-UnSeeN (7 chunks scored)



Processing 2005_Hitch.2005.WS.DVDRiP.XViD-CCTV.en:   0%|  | 0/9 [00:00<?, ?it/s]
Processing 2005_Hitch.2005.WS.DVDRiP.XViD-CCTV.en:  11%| | 1/9 [00:03<00:25,  3.
Processing 2005_Hitch.2005.WS.DVDRiP.XViD-CCTV.en:  22%|▏| 2/9 [00:06<00:21,  3.
Processing 2005_Hitch.2005.WS.DVDRiP.XViD-CCTV.en:  33%|▎| 3/9 [00:08<00:17,  2.
Processing 2005_Hitch.2005.WS.DVDRiP.XViD-CCTV.en:  44%|▍| 4/9 [00:11<00:14,  2.
Processing 2005_Hitch.2005.WS.DVDRiP.XViD-CCTV.en:  56%|▌| 5/9 [00:14<00:11,  2.
Processing 2005_Hitch.2005.WS.DVDRiP.XViD-CCTV.en:  67%|▋| 6/9 [00:17<00:08,  2.
Processing 2005_Hitch.2005.WS.DVDRiP.XViD-CCTV.en:  78%|▊| 7/9 [00:20<00:05,  2.
Processing 2005_Hitch.2005.WS.DVDRiP.XViD-CCTV.en:  89%|▉| 8/9 [00:23<00:02,  2.
Processing 2005_Hitch.2005.WS.DVDRiP.XViD-CCTV.en: 100%|█| 9/9 [00:25<00:00,  2.
Scoring movies:  18%|████▌                     | 78/443 [03:14<59:37,  9.80s/it]

✅ Finished: 2005_Hitch.2005.WS.DVDRiP.XViD-CCTV.en (9 chunks scored)



Processing 1983_Flashdance.[Flashdance].1983.DVDRip.XviD.telumethar.eDonkers.cd1
Processing 1983_Flashdance.[Flashdance].1983.DVDRip.XviD.telumethar.eDonkers.cd1
Processing 1983_Flashdance.[Flashdance].1983.DVDRip.XviD.telumethar.eDonkers.cd1
Scoring movies:  18%|████▌                     | 78/443 [03:23<15:49,  2.60s/it]


KeyboardInterrupt: 

In [5]:
import os
import json
import time
import requests
from pathlib import Path
from tqdm import tqdm

# === API setup ===
api_key = "apikey"  # 👈 Insert your API key here
api_url = "https://api.anthropic.com/v1/messages"
model_name = "claude-3-haiku-20240307"

headers = {
    "x-api-key": api_key,
    "anthropic-version": "2023-06-01",
    "content-type": "application/json",
}

# === Paths ===
base_path = Path.home() / "Desktop" / "processed_subs"
prompts_dir = base_path / "prompts_claude_new"
output_dir = base_path / "scored_prompts_claude_new"
output_dir.mkdir(parents=True, exist_ok=True)

# === Helper: call Claude ===
def call_claude(prompt):
    payload = {
        "model": model_name,
        "max_tokens": 512,
        "temperature": 0,
        "system": "You are a political sentiment scorer. Always respond ONLY with the exact JSON format requested.",
        "messages": [{"role": "user", "content": prompt}]
    }
    
    response = requests.post(api_url, headers=headers, json=payload)
    
    if response.status_code == 429:
        print(f"⚠️ Rate limit hit. Waiting 60 seconds...")
        time.sleep(60)
        return call_claude(prompt)  # Retry once after waiting
    
    if response.status_code != 200:
        print(f"⚠️ API Error {response.status_code}: {response.text}")
        return None
    
    data = response.json()
    content = data["content"][0]["text"]
    
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        print(f"⚠️ JSON parsing error: {content}")
        return None

# === Main Scoring Loop ===
prompt_files = list(prompts_dir.glob("*.json"))
print(f"Found {len(prompt_files)} prompt files.")

for prompt_file in tqdm(prompt_files, desc="Scoring movies"):
    filename_stem = prompt_file.stem.replace("_prompts", "")
    output_path = output_dir / f"{filename_stem}_scored.json"
    
    with open(prompt_file, "r", encoding="utf-8") as f:
        prompts_data = json.load(f)
    
    # Check if already scored and completed
    if output_path.exists():
        with open(output_path, "r", encoding="utf-8") as f:
            existing_scored = json.load(f)
        
        if len(existing_scored) >= len(prompts_data):
            print(f"✅ {filename_stem} already fully scored, skipping.")
            continue
        else:
            print(f"🔄 {filename_stem} partially scored, resuming...")
    else:
        existing_scored = []
    
    # Resume from the last chunk scored
    scored_chunks = existing_scored.copy()
    start_idx = len(existing_scored)
    
    for entry in tqdm(prompts_data[start_idx:], desc=f"Processing {filename_stem}", leave=False):
        chunk_id = entry["chunk_id"]
        prompt = entry["prompt"]
        
        result = call_claude(prompt)
        
        if result is None:
            print(f"❌ Failed on chunk {chunk_id} in {filename_stem}")
            continue
        
        try:
            scored_chunks.append({
                "chunk_id": chunk_id,
                "universalism_score": result["universalism"][0],
                "universalism_confidence": result["universalism"][1],
                "egalitarianism_score": result["egalitarianism"][0],
                "egalitarianism_confidence": result["egalitarianism"][1],
                "progress_score": result["progress"][0],
                "progress_confidence": result["progress"][1],
            })
        except Exception as e:
            print(f"⚠️ Error parsing chunk {chunk_id}: {e}")
            continue
        
        time.sleep(1.5)  # Respect rate limits

        # Save progress after each chunk (to prevent loss on crash)
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(scored_chunks, f, indent=2)

    print(f"✅ Finished scoring: {filename_stem}")

print("\n🎉 All done! Movies scored without missing chunks!")


Found 443 prompt files.


Scoring movies:   0%|                                   | 0/443 [00:00<?, ?it/s]

✅ 1977_Looking.For.Mr.Goodbar.ENG.PAL.en already fully scored, skipping.
✅ 1983_Octopussy.1983.DVD5.720p.HDTV.x264-NBS.eng already fully scored, skipping.
✅ 2020_Birds.of.Prey.And.the.Fantabulous.Emancipation.of.One.Harley.Quinn.2020.HC.720p.HDRip.800M already fully scored, skipping.
✅ 1978_Every.Which.Way.But.Loose.1978.1080p.BluRay.x264-aAF.ENG already fully scored, skipping.
✅ 2002_Signs.2002.720p.BluRay.x264.DTS-HDChina.eng already fully scored, skipping.
🔄 1999_Runaway Bride (1999) [BluRay] [1080p] [YTS.AM].eng partially scored, resuming...



Processing 1999_Runaway Bride (1999) [BluRay] [1080p] [YTS.AM].eng:   0%| | 0/4 
Processing 1999_Runaway Bride (1999) [BluRay] [1080p] [YTS.AM].eng:  25%|▎| 1/4 
Processing 1999_Runaway Bride (1999) [BluRay] [1080p] [YTS.AM].eng:  50%|▌| 2/4 
Processing 1999_Runaway Bride (1999) [BluRay] [1080p] [YTS.AM].eng:  75%|▊| 3/4 
Processing 1999_Runaway Bride (1999) [BluRay] [1080p] [YTS.AM].eng: 100%|█| 4/4 
Scoring movies:   1%|▎                          | 6/443 [00:11<13:56,  1.91s/it]

✅ Finished scoring: 1999_Runaway Bride (1999) [BluRay] [1080p] [YTS.AM].eng
🔄 1986_Ferris.Buellers.Day.Off.1986.1080p.BluRay.x264-CiNEFiLE.ENG partially scored, resuming...


...

✅ Finished scoring: 2021_Ghostbusters.Afterlife.2021.1080p.AMZN.WEB-DL.DDP5.1.H.264-EVO



Processing 2006_Over.The.Hedge[2006]DvDrip.AC3[Eng]-aXXo:   0%| | 0/4 [00:00<?, 
Processing 2006_Over.The.Hedge[2006]DvDrip.AC3[Eng]-aXXo:  25%|▎| 1/4 [00:02<00:
Processing 2006_Over.The.Hedge[2006]DvDrip.AC3[Eng]-aXXo:  50%|▌| 2/4 [00:05<00:
Processing 2006_Over.The.Hedge[2006]DvDrip.AC3[Eng]-aXXo:  75%|▊| 3/4 [00:08<00:
Processing 2006_Over.The.Hedge[2006]DvDrip.AC3[Eng]-aXXo: 100%|█| 4/4 [00:10<00:
Scoring movies: 100%|██████████████████████▉| 442/443 [3:20:40<00:28, 28.37s/it]

✅ Finished scoring: 2006_Over.The.Hedge[2006]DvDrip.AC3[Eng]-aXXo



Processing 1987_Fatal Attraction 1987 1080p BluRay Remux AVC TrueHD 5. 1 COMM-KE
Processing 1987_Fatal Attraction 1987 1080p BluRay Remux AVC TrueHD 5. 1 COMM-KE
Processing 1987_Fatal Attraction 1987 1080p BluRay Remux AVC TrueHD 5. 1 COMM-KE
Processing 1987_Fatal Attraction 1987 1080p BluRay Remux AVC TrueHD 5. 1 COMM-KE
Processing 1987_Fatal Attraction 1987 1080p BluRay Remux AVC TrueHD 5. 1 COMM-KE
Processing 1987_Fatal Attraction 1987 1080p BluRay Remux AVC TrueHD 5. 1 COMM-KE
Processing 1987_Fatal Attraction 1987 1080p BluRay Remux AVC TrueHD 5. 1 COMM-KE
Processing 1987_Fatal Attraction 1987 1080p BluRay Remux AVC TrueHD 5. 1 COMM-KE
Scoring movies: 100%|███████████████████████| 443/443 [3:20:59<00:00, 27.22s/it]

✅ Finished scoring: 1987_Fatal Attraction 1987 1080p BluRay Remux AVC TrueHD 5. 1 COMM-KESH. EN 23. 976

🎉 All done! Movies scored without missing chunks!
